In [1]:
import xarray as xr
import numpy as np
import s3fs
import zarr
import pandas as pd
import dask.array as da

from zarr.storage import FSStore

### load data sampel

In [13]:
%%time
inawavesampel = xr.open_zarr("/data/local/ofs/inaflows_combined.zarr")
inaflowsampel = xr.open_zarr("/data/local/ofs/inaflows_combined.zarr")

CPU times: user 3.23 s, sys: 2.86 s, total: 6.08 s
Wall time: 22.2 s


In [2]:
inawavesampel = xr.open_dataset("/data/share/output/netcdf/ww3gfs/2025/01/w3g_hires_20250121_0000.nc")
inawavesampel

<xarray.Dataset> Size: 4GB
Dimensions:  (lon: 881, lat: 481, time: 81)
Coordinates:
  * lon      (lon) float64 7kB 90.0 90.06 90.12 90.19 ... 144.9 144.9 145.0
  * lat      (lat) float64 4kB -15.0 -14.94 -14.88 -14.81 ... 14.88 14.94 15.0
  * time     (time) datetime64[ns] 648B 2025-01-21 ... 2025-01-31
Data variables: (12/17)
    dtn      (time) float64 648B ...
    hs       (time, lat, lon) float64 275MB ...
    lm       (time, lat, lon) float64 275MB ...
    t01      (time, lat, lon) float64 275MB ...
    dir      (time, lat, lon) float64 275MB ...
    phs00    (time, lat, lon) float64 275MB ...
    ...       ...
    pdi00    (time, lat, lon) float64 275MB ...
    pdi01    (time, lat, lon) float64 275MB ...
    pdi02    (time, lat, lon) float64 275MB ...
    uwnd     (time, lat, lon) float64 275MB ...
    vwnd     (time, lat, lon) float64 275MB ...
    hmax     (time, lat, lon) float64 275MB ...
Attributes:
    source:       Inawaves - BMKG Ocean Forecast System (OFS)
    description:  Inawaves Model - Forecast
    institution:  BMKG - Center For Marine Meteorology
    email:        produksi.maritim@bmkg.go.id
    Conventions:  CF-1.8

In [2]:
%%time
# cawosampel = xr.open_dataset("/data/raid0/netcdf/inaCAWO/2025/01/InaCAWOWave_20250101_1200.nc").isel(time=slice(0,1))
cawowave = xr.open_dataset("/data/raid0/netcdf/inaCAWO/2025/01/InaCAWOWave_20250101_1200.nc").isel(time=0)
cawoflow = xr.open_dataset("/data/raid0/netcdf/inaCAWO/2025/01/InaCAWOFlows_20250101_1200.nc").isel(time=0, depth=0)
cawoatmo = xr.open_dataset("/data/raid0/netcdf/inaCAWO/2025/01/InaCAWOAtmos_20250102_1200.nc").isel(time=0)
cawosampel = xr.merge([cawowave, cawoflow, cawoatmo], compat='override')
cawosampel

CPU times: user 179 ms, sys: 71.4 ms, total: 250 ms
Wall time: 409 ms


<xarray.Dataset> Size: 264MB
Dimensions:  (lat: 1201, lon: 2201)
Coordinates:
    time     datetime64[ns] 8B 2025-01-01T12:00:00
  * lat      (lat) float64 10kB -15.0 -14.97 -14.95 -14.92 ... 14.95 14.98 15.0
  * lon      (lon) float64 18kB 90.0 90.02 90.05 90.08 ... 144.9 145.0 145.0
    depth    float64 8B 0.0
Data variables: (12/25)
    hs       (lat, lon) float32 11MB ...
    hmax     (lat, lon) float32 11MB ...
    lm       (lat, lon) float32 11MB ...
    t01      (lat, lon) float32 11MB ...
    dir      (lat, lon) float32 11MB ...
    phs00    (lat, lon) float32 11MB ...
    ...       ...
    W        (lat, lon) float32 11MB ...
    mslp     (lat, lon) float32 11MB ...
    t        (lat, lon) float32 11MB ...
    td       (lat, lon) float32 11MB ...
    rh       (lat, lon) float32 11MB ...
    ch       (lat, lon) float32 11MB ...
Attributes:
    source:       InaCAWO Wave - 3km High Resolution
    description:  Indonesia Coupled Atmosphere Wave Ocean (InaCAWO) Wave - Fo...
    institution:  BMKG - Center For Marine Meteorology
    contact:      produksi.maritim@bmkg.go.id

### expand time dimension

In [7]:
new_time = pd.date_range("2024-01-01", "2025-04-01", freq="1H", inclusive="left")

# Buat shell kosong
data_vars = {}
for var in cawosampel.data_vars:
    # Dims baru (pastikan 'time' di depan)
    original_dims = cawosampel[var].dims
    new_dims = ("time",) + tuple(dim for dim in original_dims if dim != "time")

    # Shape baru sesuai new_time
    new_shape = (len(new_time),) + tuple(cawosampel[var].sizes[dim] for dim in original_dims if dim != "time")

    # Ambil hanya koordinat non-'time' yang ada di dims
    coords = {
        dim: cawosampel.coords[dim]
        for dim in new_dims
        if dim in cawosampel.coords and dim != "time"
    }

    # Buat array kosong (belum isi NaN)
    data = xr.DataArray(
        da.empty(new_shape, dtype=cawosampel[var].dtype, chunks=((24,) + new_shape[1:])),
        dims=new_dims,
        coords=coords,
        attrs=cawosampel[var].attrs
    )
    data_vars[var] = data

# Buat dataset final
cawosampelnan = xr.Dataset(data_vars)
cawosampelnan = cawosampelnan.assign_coords(time=new_time)
cawosampelnan.attrs['source '] = 'InaCAWO - 3km High Resolution'
cawosampelnan.attrs['description '] = 'Indonesia Coupled Atmosphere Wave Ocean (InaCAWO)'
cawosampelnan.attrs['institution '] = 'BMKG - Center For Marine Meteorology'
cawosampelnan.attrs['contact '] = 'produksi.maritim@bmkg.go.id'
cawosampelnan.attrs['conventions '] = 'CF-1.8'

cawosampelnan

/tmp/ipykernel_171/2717298669.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  new_time = pd.date_range("2024-01-01", "2025-04-01", freq="1H", inclusive="left")


<xarray.Dataset> Size: 3TB
Dimensions:  (lat: 1201, lon: 2201, time: 10944)
Coordinates:
  * lat      (lat) float64 10kB -15.0 -14.97 -14.95 -14.92 ... 14.95 14.98 15.0
  * lon      (lon) float64 18kB 90.0 90.02 90.05 90.08 ... 144.9 145.0 145.0
  * time     (time) datetime64[ns] 88kB 2024-01-01 ... 2025-03-31T23:00:00
Data variables: (12/25)
    hs       (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    hmax     (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    lm       (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    t01      (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    dir      (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    phs00    (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    ...       ...
    W        (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    mslp     (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    t        (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    td       (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    rh       (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    ch       (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
Attributes:
    source :       InaCAWO - 3km High Resolution
    description :  Indonesia Coupled Atmosphere Wave Ocean (InaCAWO)
    institution :  BMKG - Center For Marine Meteorology
    contact :      produksi.maritim@bmkg.go.id
    conventions :  CF-1.8

In [8]:
print(type(cawosampelnan["hs"].data))


<class 'dask.array.core.Array'>


In [11]:
cawosampelnan[var].shape[1:]

(1201, 2201)

### empty zarr initialization

In [13]:
%%time
import numcodecs
import s3fs

fs_options = {
    "key": "moqs1u5xKGk6tRx3GbyJ",
    "secret": "OLGsgagMmgPWE6aRaBGdiQUC7U4WLFKQXIGiN821",
    "client_kwargs": {
        "endpoint_url": "http://172.17.0.1:9990"
    }
}

# Setup filesystem MinIO via fsspec
fs = s3fs.S3FileSystem(
    anon=False,
    key=fs_options["key"],
    secret=fs_options["secret"],
    client_kwargs=fs_options["client_kwargs"]
)

# Path di dalam bucket
zarr_path = "zarr-data/combined"

# Encoding: define chunking + compressor
encoding = {
    var: {
        "chunks": (24,) + (cawosampelnan[var].shape[1:]),  # chunk harian
        "compressor": numcodecs.Blosc(cname="zstd", clevel=3)
    }
    for var in cawosampelnan.data_vars
}

import logging
from dask.diagnostics import ProgressBar

logging.basicConfig(level=logging.INFO)
ProgressBar().register()

# Simpan ke Zarr di object storage
cawosampelnan.to_zarr(
    store=fs.get_mapper(zarr_path),
    mode="w",
    encoding=encoding,
    consolidated=True
)


[########################################] | 100% Completed | 164.48 s
CPU times: user 45min 8s, sys: 20min 14s, total: 1h 5min 22s
Wall time: 2min 50s


> Percobaan chunking 24, 601, dan 1101 prosesnya 6x lebih lama. Chunk paling optimal 24, 1201, 2201.

### add cycle

In [14]:
import pandas as pd
import numpy as np
import xarray as xr
import s3fs

fs_options = {
    "key": "moqs1u5xKGk6tRx3GbyJ",
    "secret": "OLGsgagMmgPWE6aRaBGdiQUC7U4WLFKQXIGiN821",
    "client_kwargs": {
        "endpoint_url": "http://172.17.0.1:9990"
    }
}

fs = s3fs.S3FileSystem(
    anon=False,
    key=fs_options["key"],
    secret=fs_options["secret"],
    client_kwargs=fs_options["client_kwargs"]
)

zarr_path = "zarr-data/combined"
mapper = fs.get_mapper(zarr_path)

# Buka metadata zarr tanpa load data
ds = xr.open_zarr(mapper, consolidated=True)

# Hitung waktu produksi (cycle) berdasarkan jam time
def infer_cycle(time_index):
    cycle_hours = [0, 12]
    return pd.to_datetime([
        pd.Timestamp(t).floor('1D') + pd.Timedelta(hours=max(h for h in cycle_hours if h <= pd.Timestamp(t).hour))
        for t in time_index
    ])

cycle_array = infer_cycle(ds.time.values)

# Buat dataset hanya berisi variabel cycle
cycle_ds = xr.Dataset({
    "cycle": ("time", cycle_array)
})
cycle_ds["cycle"].attrs["description"] = "Production cycle datetime (e.g., 00, 06, 12, 18 UTC)"
cycle_ds

<xarray.Dataset> Size: 88kB
Dimensions:  (time: 10944)
Dimensions without coordinates: time
Data variables:
    cycle    (time) datetime64[ns] 88kB 2024-01-01 ... 2025-03-31T12:00:00

In [15]:
cycle_ds.cycle[:20]

<xarray.DataArray 'cycle' (time: 20)> Size: 160B
array(['2024-01-01T00:00:00.000000000', '2024-01-01T00:00:00.000000000',
       '2024-01-01T00:00:00.000000000', '2024-01-01T00:00:00.000000000',
       '2024-01-01T00:00:00.000000000', '2024-01-01T00:00:00.000000000',
       '2024-01-01T00:00:00.000000000', '2024-01-01T00:00:00.000000000',
       '2024-01-01T00:00:00.000000000', '2024-01-01T00:00:00.000000000',
       '2024-01-01T00:00:00.000000000', '2024-01-01T00:00:00.000000000',
       '2024-01-01T12:00:00.000000000', '2024-01-01T12:00:00.000000000',
       '2024-01-01T12:00:00.000000000', '2024-01-01T12:00:00.000000000',
       '2024-01-01T12:00:00.000000000', '2024-01-01T12:00:00.000000000',
       '2024-01-01T12:00:00.000000000', '2024-01-01T12:00:00.000000000'],
      dtype='datetime64[ns]')
Dimensions without coordinates: time
Attributes:
    description:  Production cycle datetime (e.g., 00, 06, 12, 18 UTC)

In [16]:
encoding = {
    "cycle": {
        "chunks": (24,),  # atau sama seperti 'time'
        "dtype": "datetime64[ns]"
    }
}

cycle_ds.to_zarr(
    store=mapper,
    mode="a",               # Tambahkan ke existing Zarr
    encoding=encoding,
    consolidated=False      # Karena kita hanya update, tidak perlu konsolidasikan metadata lagi
)


In [17]:
zarr.consolidate_metadata(store=mapper) # necessary

<zarr.hierarchy.Group '/'>

In [18]:
True

True

### open dataset

In [19]:
import s3fs
import xarray as xr

fs_options = {
    "key": "moqs1u5xKGk6tRx3GbyJ",
    "secret": "OLGsgagMmgPWE6aRaBGdiQUC7U4WLFKQXIGiN821",
    "client_kwargs": {
        "endpoint_url": "http://172.17.0.1:9990"
    }
}

fs = s3fs.S3FileSystem(
    anon=False,
    key=fs_options["key"],
    secret=fs_options["secret"],
    client_kwargs=fs_options["client_kwargs"]
)

# Path ke Zarr dataset di dalam bucket
zarr_path = "zarr-data/combined"
mapper = fs.get_mapper(zarr_path)
ds_attrs_only = xr.Dataset()
ds_attrs_only.attrs = {
    "source": "InaCAWO - 3km High Resolution",
    "description": "Indonesia Coupled Atmosphere Wave Ocean (InaCAWO)",
    "institution": "BMKG - Center For Marine Meteorology",
    "contact": "produksi.maritim@bmkg.go.id",
    "conventions": "CF-1.8"
}

ds_attrs_only.to_zarr(
    store=fs.get_mapper(zarr_path),
    mode="a",
    consolidated=False
)

import zarr
zarr.consolidate_metadata(fs.get_mapper(zarr_path))

<zarr.hierarchy.Group '/'>

In [20]:
import s3fs
import xarray as xr

fs_options = {
    "key": "moqs1u5xKGk6tRx3GbyJ",
    "secret": "OLGsgagMmgPWE6aRaBGdiQUC7U4WLFKQXIGiN821",
    "client_kwargs": {
        "endpoint_url": "http://172.17.0.1:9990"
    }
}

fs = s3fs.S3FileSystem(
    anon=False,
    key=fs_options["key"],
    secret=fs_options["secret"],
    client_kwargs=fs_options["client_kwargs"]
)

# Path ke Zarr dataset di dalam bucket
zarr_path = "zarr-data/combined"
mapper = fs.get_mapper(zarr_path)

ds = xr.open_zarr(mapper, consolidated=False)
ds


<xarray.Dataset> Size: 3TB
Dimensions:  (time: 10944, lat: 1201, lon: 2201)
Coordinates:
  * lat      (lat) float64 10kB -15.0 -14.97 -14.95 -14.92 ... 14.95 14.98 15.0
  * lon      (lon) float64 18kB 90.0 90.02 90.05 90.08 ... 144.9 145.0 145.0
  * time     (time) datetime64[ns] 88kB 2024-01-01 ... 2025-03-31T23:00:00
Data variables: (12/26)
    S        (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    T        (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    W        (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    ch       (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    cycle    (time) datetime64[ns] 88kB dask.array<chunksize=(24,), meta=np.ndarray>
    dir      (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    ...       ...
    td       (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    u        (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    uwnd     (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    v        (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    vwnd     (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
    zeta     (time, lat, lon) float32 116GB dask.array<chunksize=(24, 1201, 2201), meta=np.ndarray>
Attributes:
    contact:      produksi.maritim@bmkg.go.id
    conventions:  CF-1.8
    description:  Indonesia Coupled Atmosphere Wave Ocean (InaCAWO)
    institution:  BMKG - Center For Marine Meteorology
    source:       InaCAWO - 3km High Resolution

In [25]:
livar = []
for var in list(ds):
    try:
        livar.append(ds[var].attrs['standard_name'])
    except:
        continue
livar

['sea_water_salinity',
 'sea_water_potential_temperature',
 'upward_sea_water_velocity',
 'precipitation_amount',
 'sea_surface_mean_wave_from_direction',
 'GUST_SPEED_10M',
 'sea_surface_wave_maximum_height',
 'sea_surface_wave_significant_height',
 'sea_surface_mean_wave_length',
 'air_pressure_at_mean_sea_level',
 'sea_surface_wind_wave_from_direction',
 'sea_surface_primary_swell_wave_from_direction',
 'sea_surface_wind_wave_significant_height',
 'sea_surface_primary_swell_wave_significant_height',
 'sea_surface_wind_wave_mean_period',
 'sea_surface_primary_swell_wave_mean_period',
 'relative_humidity',
 'air_temperature',
 'sea_surface_wave_peak_period',
 'dew_point_temperature',
 'eastward_sea_water_velocity',
 'eastward_wind',
 'northward_sea_water_velocity',
 'northward_wind',
 'sea_surface_height_above_mean_sea_level']

### update zarr

In [ ]:
def update_zarr_forecast(ds_new, zarr_path, fs_kwargs, time_dim="time"):
    fs = s3fs.S3FileSystem(**fs_kwargs)
    store = FSStore(zarr_path, fs=fs)

    t_new = ds_new[time_dim].values

    if not fs.exists(zarr_path):
        print("Zarr belum ada, inisialisasi...")
        ds_new.to_zarr(store=store, mode="w")
        return

    print("Membuka Zarr existing...")
    root = zarr.open_group(store, mode="a")
    z_meta = zarr.open_group(store, mode="r")  # read metadata via xarray
    t_existing = z_meta[time_dim][:]
    
    overlap_mask = np.isin(t_new, t_existing)
    new_mask = ~overlap_mask
    print(f"  Replace: {overlap_mask.sum()} | Append: {new_mask.sum()}")

    for var in ds_new.data_vars:
        zarr_array = root[var]
        data_new = ds_new[var].values

        for i, t in enumerate(t_new):
            if overlap_mask[i]:
                idx_existing = np.where(t_existing == t)[0][0]
                zarr_array[idx_existing] = data_new[i]

    if new_mask.any():
        ds_append = ds_new.isel({time_dim: np.where(new_mask)[0]})
        append_len = ds_append.dims[time_dim]
        
        for var in ds_new.data_vars:
            zarr_array = root[var]
            old_shape = zarr_array.shape[0]
            new_shape = old_shape + append_len

            print(f"  Resize var {var}: {old_shape} → {new_shape}")
            zarr_array.resize(new_shape, axis=0)
            zarr_array[old_shape:new_shape] = ds_append[var].values

        zarr_time = root[time_dim]
        zarr_time.resize(new_shape)
        zarr_time[old_shape:new_shape] = ds_append[time_dim].values

    print("✅ Update Zarr selesai.")

In [ ]:
fs_options = {
    "key": "moqs1u5xKGk6tRx3GbyJ",
    "secret": "OLGsgagMmgPWE6aRaBGdiQUC7U4WLFKQXIGiN821",
    "client_kwargs": {
        "endpoint_url": "172.17.0.1:9990"
    }
}

update_zarr_forecast(ds_new, "zarr-data/model-archive.zarr", fs_options)